In [0]:
import dataiku
import io
import pandas as pd
import re
import os
import json

# Définir les dossiers d'entrée et de sortie
pdf_folder = dataiku.Folder("W8lS5GmB")  # Dossier contenant les PDF originaux
md_folder = dataiku.Folder("d7DdDueY")   # Dossier contenant les annotations générées

# Lister les fichiers PDF
pdf_files = [f for f in pdf_folder.list_paths_in_partition() if f.lower().endswith(".pdf")]
pdf_files.sort()

# Fonction pour lire le contenu d'un fichier s'il existe
def read_file_content(folder, file_path):
    if file_path in folder.list_paths_in_partition():
        with folder.get_download_stream(file_path) as stream:
            return io.BytesIO(stream.read()).read().decode("utf-8")
    return None

# Initialiser le dictionnaire pour stocker les données
data = {
    "doc": [],             # Nom complet du PDF avec la page
    "doc_root": [],        # Nom du PDF sans la page
    "json": [],            # Contenu JSON
    "md": [],              # Contenu Markdown
    "md_img": [],          # Contenu Markdown avec descriptions d'images
    "json_img": []         # Contenu JSON avec descriptions d'images
}

# Pour chaque PDF, extraire toutes les annotations associées
for pdf_file in pdf_files:
    # Extraire le nom de base du document (sans l'extension)
    base_name = os.path.splitext(pdf_file)[0]
    doc_root = base_name.split('_page_')[0] if '_page_' in base_name else base_name
    
    # Chemins des fichiers d'annotation
    json_file = base_name + ".json"
    md_file = base_name + ".md"
    md_img_file = base_name + "__with_img_desc.md"
    json_img_file = base_name + "__with_img_desc.json"
    
    # Lire le contenu des fichiers d'annotation
    json_content = read_file_content(md_folder, json_file)
    md_content = read_file_content(md_folder, md_file)
    md_img_content = read_file_content(md_folder, md_img_file)
    json_img_content = read_file_content(md_folder, json_img_file)
    
    # Identifier les images associées au document
    img_pattern = re.compile(rf"^{re.escape(base_name)}-img-(\d+)\.jpeg$")
    all_files = md_folder.list_paths_in_partition()
    
    # Créer un dictionnaire pour stocker les informations sur les images
    images = {}
    
    # Chercher toutes les images et leurs descriptions
    for file_path in all_files:
        img_match = img_pattern.match(file_path)
        if img_match:
            img_num = img_match.group(1)
            img_key = f"img-{img_num}"
            desc_file = f"{base_name}-img-{img_num}.md"
            
            # Ajouter l'image et sa description au dictionnaire
            if img_key not in images:
                images[img_key] = {
                    "path": file_path,
                    "desc": None
                }
            
            # Vérifier si une description existe pour cette image
            if desc_file in all_files:
                img_desc = read_file_content(md_folder, desc_file)
                images[img_key]["desc"] = img_desc
    
    # Ajouter les données de base du document
    row = {
        "doc": pdf_file,
        "doc_root": doc_root + ".pdf",
        "json": json_content,
        "md": md_content,
        "md_img": md_img_content,
        "json_img": json_img_content
    }
    
    # Ajouter les informations sur les images
    for img_key, img_info in sorted(images.items()):
        row[img_key] = img_info["path"]
        if img_info["desc"]:
            row[f"{img_key}-desc"] = img_info["desc"]
    
    # Ajouter la ligne au dataset
    for key in data:
        if key in row:
            data[key].append(row[key])
        else:
            data[key].append(None)
    
    # Ajouter dynamiquement les colonnes d'images
    for key in row:
        if key not in data:
            data[key] = [None] * (len(data["doc"]) - 1)
            data[key].append(row[key])

# Créer le DataFrame
df = pd.DataFrame.from_dict(data)

# Écrire le DataFrame dans un dataset
dataiku.Dataset("a220_tech_docs_annotations").write_with_schema(df)

print(f"Extraction terminée avec succès. {len(df)} documents traités.")